# I M P L I C I T - Q - L E A R N I N G - IQN

In [1]:
import torch
torch.set_float32_matmul_precision('highest')
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset

from torch.utils.tensorboard import SummaryWriter
import numpy as np

import h5py
import tqdm


### D E V I C E

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device: {device}')


Device: cuda


### D A T A 

In [3]:
file = 'C:\OFFLINE RL\hopper_medium-v2.hdf5'

with h5py.File(file, mode = 'r') as f:
    
    observations = np.array(f['observations'])
    actions = np.array(f['actions'])
    rewards = np.array(f['rewards'])
    next_obs = np.array(f['next_observations'])
    
state_dim = observations.shape[1]
action_dim = actions.shape[1]
max_action = abs(actions).max()

print(f'len of data: {len(observations)}')
print(f'state dim: {state_dim} | action dim: {action_dim} | max action: {max_action}')


len of data: 1000000
state dim: 11 | action dim: 3 | max action: 0.9999945163726807


### D A T A - E N G I N E E R I N G

In [4]:
obs_tensor = torch.from_numpy(observations).float().to(device)
act_tensor = torch.from_numpy(actions).float().to(device)
rew_tensor = torch.from_numpy(rewards).float().to(device)
next_obs_tensor = torch.from_numpy(next_obs).float().to(device)

class Hopper_Dataset(Dataset):
    
    def __init__(self, obs, act, rew, next_obs_t):
        
        self.states = obs
        self.actions = act
        self.rewards = rew
        self.next_states = next_obs_t
        
    def __len__(self):
        
        return len(self.states)
    
    def __getitem__(self, index):
        return self.states[index], self.actions[index], self.rewards[index], self.next_states[index]
    
dataset = Hopper_Dataset(obs_tensor, act_tensor, rew_tensor, next_obs_tensor)

train_data, test_data = train_test_split(dataset, test_size = 0.1, shuffle = False)

train_loader, test_loader = DataLoader(train_data, batch_size = 256, shuffle = True, drop_last = True), DataLoader(test_data, batch_size = 256, shuffle = True, drop_last = True)


### L O G G I N G

In [5]:
writer = SummaryWriter(log_dir = './runs/IQL')


### A S S E M B L Y

In [6]:
head_1 = 128
head_2 = 256
head_3 = 256
head_4 = 128

hidden_size = 128
hidden_size_2 = 256


### F E A T U R E 

In [7]:
class Feature_Extractor(nn.Module):
    
    def __init__(self, input_dim, output_dim, hidden_size = hidden_size, hidden_size_2 = hidden_size_2):
        super(Feature_Extractor, self).__init__()
        
        self.cal = nn.Sequential(
            
            nn.Linear(input_dim, hidden_size),
            nn.SiLU(),
            
            nn.LayerNorm(hidden_size),
            nn.Linear(hidden_size, hidden_size_2),
            nn.SiLU(),
            
            nn.Linear(hidden_size_2, hidden_size),
            nn.SiLU(),
            
            nn.Linear(hidden_size, output_dim),
            nn.SiLU()
        )
        
    def forward(self, x):
    
        return self.cal(x)


### P O L I C Y

In [8]:
class policy_net(nn.Module):
    
    def __init__(self, head_1 = head_1, head_2 = head_2, head_3 = head_3, head_4 = head_4, max_action = max_action, state_dim = state_dim, action_dim = action_dim):
        super(policy_net, self).__init__()
        
        # feature 
        
        self.feature = Feature_Extractor(state_dim, head_1)
        
        # norm
        
        self.norm  = nn.LayerNorm(head_1)
        
        # pos process
        
        self.pos_process = nn.Sequential(
            
            nn.Linear(head_1, head_2),
            nn.SiLU(),
            
            nn.Linear(head_2, head_3),
            nn.SiLU(),
            
            nn.Linear(head_3, head_4),
            nn.SiLU()
        ) 
        
        # mu and log std head
        
        self.mu = nn.Linear(head_4, action_dim)
        self.log_std = nn.Linear(head_4, action_dim)
        
        # max action
        
        self.max_action = max_action
        
    def forward(self, state, deterministic = False, return_log_probs = False):
        
        # feature
        
        feature = self.feature(state)
        
        # norm
        
        norm = self.norm(feature)
        
        # pos
        
        pos = self.pos_process(norm)
        
        # mu and log std
        
        mu = self.mu(pos)
        log_std = self.log_std(pos)
        
        log_std = torch.clamp(log_std, -10, 2)
        std = torch.exp(log_std)
        
        dist = torch.distributions.Normal(mu, std)
        z = mu if deterministic == True else dist.rsample()
        tanh_z = torch.tanh(z)
        action = tanh_z * self.max_action
        
        log_prob = None
        
        if return_log_probs:
            
            log_prob = dist.log_prob(z).sum(dim = -1, keepdim = True)
            log_prob -= torch.log(1 - tanh_z.pow(2) + 1e-6).sum(dim = -1, keepdim = True)
        
        return action, mu, log_std, log_prob
        

### V A L U E 

In [9]:
class value_net(nn.Module):
    
    def __init__(self, state_dim = state_dim, head_1 = head_1, head_2 = head_2, head_3 = head_3, head_4 = head_4):
        super(value_net, self).__init__()
        
        # process
        
        self.process = nn.Sequential(
            
            nn.Linear(state_dim, head_1),
            nn.SiLU(),
            
            nn.Linear(head_1, head_2),
            nn.SiLU(),
            
            
            nn.LayerNorm(head_2),
            nn.Linear(head_2, head_3),
            nn.SiLU(),
            
            
            nn.LayerNorm(head_3),
            nn.Linear(head_3, head_4),
            nn.SiLU(),
            
            nn.Linear(head_4, 1)
        )
        
    def forward(self, state):
        
        value = self.process(state)
        
        return value


### Q - F U N C T I O N 

In [10]:
class q_net(nn.Module):
    
    def __init__(self, state_dim = state_dim, action_dim = action_dim, head_1 = head_1, head_2 = head_2, head_3 = head_3, head_4 = head_4):
        super(q_net, self).__init__()
        
        self.q_1 = nn.Sequential(
            
            nn.Linear(state_dim + action_dim, head_1),
            nn.SiLU(),
            
            nn.Linear(head_1, head_2),
            nn.SiLU(),
            
            nn.LayerNorm(head_2),
            nn.Linear(head_2, head_3),
            nn.SiLU(),
            
            nn.LayerNorm(head_3),
            nn.Linear(head_3, head_4),
            nn.SiLU(),
            
            nn.Linear(head_4, 1)
        )
        
        self.q_2 = nn.Sequential(
            
            nn.Linear(state_dim + action_dim, head_1),
            nn.SiLU(),
            
            nn.Linear(head_1, head_2),
            nn.SiLU(),
            
            nn.LayerNorm(head_2),
            nn.Linear(head_2, head_3),
            nn.SiLU(),
            
            nn.LayerNorm(head_3),
            nn.Linear(head_3, head_4),
            nn.SiLU(),
            
            nn.Linear(head_4, 1)
        )
        
    def forward(self, state, action):
        
        # cat
        
        cat = torch.cat([state, action], dim = -1)
        
        # prepare
        
        q_1 = self.q_1(cat)
        q_2 = self.q_2(cat)
        
        return q_1, q_2


### S E T U P 

In [11]:
POLICY_NETWORK = policy_net().to(device)
print(POLICY_NETWORK)
print('-' * 100)

VALUE_NETWORK = value_net().to(device)
print(VALUE_NETWORK)
print('-' * 100)

Q_NETWORK = q_net().to(device)
print(Q_NETWORK)
print('-' * 100)


policy_net(
  (feature): Feature_Extractor(
    (cal): Sequential(
      (0): Linear(in_features=11, out_features=128, bias=True)
      (1): SiLU()
      (2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      (3): Linear(in_features=128, out_features=256, bias=True)
      (4): SiLU()
      (5): Linear(in_features=256, out_features=128, bias=True)
      (6): SiLU()
      (7): Linear(in_features=128, out_features=128, bias=True)
      (8): SiLU()
    )
  )
  (norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
  (pos_process): Sequential(
    (0): Linear(in_features=128, out_features=256, bias=True)
    (1): SiLU()
    (2): Linear(in_features=256, out_features=256, bias=True)
    (3): SiLU()
    (4): Linear(in_features=256, out_features=128, bias=True)
    (5): SiLU()
  )
  (mu): Linear(in_features=128, out_features=3, bias=True)
  (log_std): Linear(in_features=128, out_features=3, bias=True)
)
---------------------------------------------------------------------------

### O P T I M I Z E R - S C H E D U L E R

In [12]:
# lr

T_max = 100
policy_lr = 1e-4
value_lr = 1e-3
q_lr = 1e-3

# O P T I M I Z E R

def get_optimizer(network, lr):
    
    return optim.AdamW(network.parameters(), lr, weight_decay = 0)

policy_optimizer = get_optimizer(POLICY_NETWORK, policy_lr)
value_optimizer = get_optimizer(VALUE_NETWORK, value_lr)
q_net_optimizer = get_optimizer(Q_NETWORK, q_lr)

# S C H E D U L E R

def get_scheduler(optimizer, T_max = T_max):
    
    return optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max, eta_min = 1e-5)

policy_scheduler = get_scheduler(policy_optimizer)
value_scheduler = get_scheduler(value_optimizer)
q_net_scheduler = get_scheduler(q_net_optimizer)


### L O S S - F U N C

In [13]:
class loss_func:
    
    def __init__(self, gamma, tau, beta, POLICY_NETWORK = POLICY_NETWORK, VALUE_NETWORK = VALUE_NETWORK, Q_NETWORK = Q_NETWORK, policy_optimizer = policy_optimizer, value_optimizer = value_optimizer, q_net_optimizer = q_net_optimizer, policy_scheduler = policy_scheduler, value_scheduler = value_scheduler, q_net_scheduler = q_net_scheduler):
        
        # network
        
        self.policy = POLICY_NETWORK
        self.value = VALUE_NETWORK
        self.q_network = Q_NETWORK
        
        # optimizer
        
        self.policy_optimizer = policy_optimizer
        self.value_optimizer = value_optimizer
        self.q_optimizer = q_net_optimizer
        
        # scheduler
        
        self.policy_scheduler = policy_scheduler
        self.value_scheduler = value_scheduler
        self.q_scheduler = q_net_scheduler
        
        # hyper params
        
        self.gamma = gamma
        self.tau = tau
        self.beta = beta
        
    def value_loss(self, states, rewards, next_states):
        
        # current v
        
        v = self.value(states)
        
        # next v
        
        with torch.no_grad():
            
            next_v = self.value(next_states)
            
        delta = rewards + self.gamma * next_v - v
        weight = torch.where(delta > 0, self.tau, 1 - self.tau)
        
        value_loss = (weight * delta.pow(2)).mean()
        
        return value_loss
    
    def q_loss(self, states, actions, rewards, next_states):
        
        # cal target value
        
        with torch.no_grad():
            
            next_v = self.value(next_states)
            target = rewards + self.gamma * next_v
            
        q1, q2 = self.q_network(states, actions)
        l1 = F.mse_loss(q1, target)
        l2 = F.mse_loss(q2, target)
        
        loss = 0.5 * (l1 + l2)
        
        return loss
    
    def policy_loss(self, states):
        
        # cal log probs and actions
        
        actions, _, _, log_probs = self.policy(states, return_log_probs = True)
        
        # cal q vals
        
        with torch.no_grad():
            
            q1, q2 = self.q_network(states, actions)
            
            q = torch.min(q1, q2)
            
        # cal v
        
            v = self.value(states)
            
            advantages = q - v
        
        # cal weights
        
        weights = torch.exp(self.beta * advantages)
        weights = torch.clamp(weights, max = 100.0)
        
        # cal policy loss
        
        policy_loss = -(weights * log_probs).mean()
        
        
        return policy_loss, advantages
    
    def update(self, data_loader, epoch):
        
        running_policy_loss = 0.0
        running_value_loss = 0.0
        running_q_loss = 0.0
        all_advantages = []
        
        
        for states, actions, rewards, next_states in data_loader:
            
            if rewards.dim() == 1: rewards = rewards.unsqueeze(1)
        
            # cal value loss
            
            value_loss = self.value_loss(states, rewards, next_states)
            running_value_loss += value_loss.item()
            
            # update value network
            
            self.value_optimizer.zero_grad()
            value_loss.backward()
            torch.nn.utils.clip_grad_norm_(self.value.parameters(), max_norm = 0.5)
            self.value_optimizer.step()
            self.value_scheduler.step()
            
            # cal q _ loss
            
            q_loss = self.q_loss(states, actions, rewards, next_states)
            running_q_loss += q_loss.item()
            
            # update q network
            
            self.q_optimizer.zero_grad()
            q_loss.backward()
            torch.nn.utils.clip_grad_norm_(self.q_network.parameters(), max_norm = 0.5)
            self.q_optimizer.step()
            self.q_scheduler.step()
            
            # policy loss
            
            policy_loss, advantages = self.policy_loss(states)
            running_policy_loss += policy_loss.item()
            all_advantages.append(advantages.detach().cpu())
            
            # update
            
            self.policy_optimizer.zero_grad()
            policy_loss.backward()
            torch.nn.utils.clip_grad_norm_(self.policy.parameters(), max_norm = 0.5)
            self.policy_optimizer.step()
            self.policy_scheduler.step()
            
        avg_value_loss = running_value_loss / len(data_loader)
        avg_q_loss = running_q_loss / len(data_loader)
        avg_policy_loss = running_policy_loss / len(data_loader)
        
        all_advantages = torch.cat(all_advantages, dim = 0)
        writer.add_histogram('Advantages', all_advantages, epoch)
        
        return avg_value_loss, avg_q_loss, avg_policy_loss


### S E T U P 


In [14]:
# hyper params

gamma = 0.97
tau = 0.7
beta = 2.0

# setup

LOSS_FUNCTION = loss_func(gamma, tau, beta)


### T R A I N I N G

In [15]:
epochs = 100

for epoch in tqdm.tqdm(range(epochs), desc = 'Training IQL'):
    
    avg_value_loss, avg_q_loss, avg_policy_loss = LOSS_FUNCTION.update(train_loader, epoch)
    
    writer.add_scalar('Value loss', avg_value_loss, epoch)
    writer.add_scalar('Q Loss', avg_q_loss, epoch)
    writer.add_scalar('Policy loss', avg_policy_loss, epoch)
    
    writer.flush()
    
    tqdm.tqdm.write(f'Epoch: {epoch} / {epochs} | '
                    f'avg v loss: {avg_value_loss:.4f} | '
                    f'avg q loss: {avg_q_loss:.4f} | '
                    f'avg policy loss: {avg_policy_loss:.4f}')
    

Training IQL:   1%|          | 1/100 [01:28<2:26:50, 89.00s/it]

Epoch: 0 / 100 | avg v loss: 1.3859 | avg q loss: 84.1923 | avg policy loss: -948.2040


Training IQL:   2%|▏         | 2/100 [02:33<2:02:00, 74.70s/it]

Epoch: 1 / 100 | avg v loss: 0.4220 | avg q loss: 26.8879 | avg policy loss: -1035.5075


Training IQL:   3%|▎         | 3/100 [03:35<1:50:57, 68.64s/it]

...

Epoch: 97 / 100 | avg v loss: 0.1120 | avg q loss: 2.9351 | avg policy loss: -397.9808


Training IQL:  99%|█████████▉| 99/100 [1:50:42<01:10, 70.31s/it]

Epoch: 98 / 100 | avg v loss: 0.1137 | avg q loss: 2.8402 | avg policy loss: -412.6640


Training IQL: 100%|██████████| 100/100 [1:52:11<00:00, 67.32s/it]

Epoch: 99 / 100 | avg v loss: 0.1150 | avg q loss: 2.9650 | avg policy loss: -380.7476
